In [1]:
import pickle as pkl
import time
import os
import numpy as np
import sys
from music21 import instrument, note, stream, chord, duration
from models.RNNAttention import create_network, sample_with_temp

import matplotlib.pyplot as plt

2021-12-06 19:32:36.618035: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-12-06 19:32:36.618053: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
# run params
BASE_DIR = os.getcwd()
music_name = 'classical'
run_folder = os.path.join(BASE_DIR,"output",music_name)

# model params
embed_size = 100
rnn_units = 256
use_attention = True

In [3]:
store_folder = os.path.join(run_folder, 'store')

with open(os.path.join(store_folder, 'distincts'), 'rb') as filepath:
    distincts = pkl.load(filepath)
    note_names, n_notes, duration_names, n_durations = distincts

with open(os.path.join(store_folder, 'lookups'), 'rb') as filepath:
    lookups = pkl.load(filepath)
    note_to_int, int_to_note, duration_to_int, int_to_duration = lookups

In [4]:
weights_folder = os.path.join(run_folder, 'weights')
weights_file = "weights-improvement-372-9.3068-bigger.h5"
model, att_model = create_network(n_notes, n_durations, embed_size, rnn_units, use_attention)

weight_source = os.path.join(weights_folder,weights_file)
model.load_weights(weight_source)

2021-12-06 19:32:41.898032: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-12-06 19:32:41.898050: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2021-12-06 19:32:41.898064: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (a0da3121b20d): /proc/driver/nvidia/version does not exist
2021-12-06 19:32:41.898172: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/usr/local/lib/python3.9/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:374: UserWarning: 

In [5]:
import random 

notes_temp = 0.75
duration_temp = 0.25
max_extra_notes = 120
max_seq_len = 128
seq_len = 16

notes_random = []
for letters in ["A","C","D","E","F","F#","G","G#"]:
    for number in range(2,8):
        notes_random.append(letters+str(number))

durations_random = [0,0.25,0.5,0.75,1,1.25,1.5,1.75,2]
sequence_length = len(notes_random)
i=1
while True:
    try:
        notes = ['C2', 'A6', 'E6', 'F#4', 'A5', 'F5', 'G#3', 'F#3', 'F6', 'D6', 'D5', 'F#5', 'G#2', 'F5', 'F#2', 'E6', 'E3', 'G3', 'C5', 'C5', 'D4', 'F#5', 'C7', 'F#3', 'G2', 'F4', 'F#3', 'D3', 'G#2', 'C5', 'A5', 'E2']
        durations = [1.5, 0.25, 0.5, 1, 0.75, 1.25, 1.5, 1.5, 0.75, 1.25, 0.5, 1.75, 0, 1, 0.25, 1.5, 1.25, 0.75, 1, 1.25, 0.5, 0.25, 1.25, 0.25, 1, 0.5, 1.25, 1, 1.75, 0.25, 1.25, 1.5]
        prediction_output = []
        notes_input_sequence = []
        durations_input_sequence = []

        overall_preds = []

        for n, d in zip(notes,durations):
            note_int = note_to_int[n]
            duration_int = duration_to_int[d]

            notes_input_sequence.append(note_int)
            durations_input_sequence.append(duration_int)

            prediction_output.append([n, d])

            if n != 'START':
                midi_note = note.Note(n)

                new_note = np.zeros(128)
                new_note[midi_note.pitch.midi] = 1
                overall_preds.append(new_note)


        att_matrix = np.zeros(shape = (max_extra_notes+sequence_length, max_extra_notes))

        for note_index in range(max_extra_notes):
            prediction_input = [
                np.array([notes_input_sequence])
                , np.array([durations_input_sequence])
               ]

            notes_prediction, durations_prediction = model.predict(prediction_input, verbose=0)
            if use_attention:
                att_prediction = att_model.predict(prediction_input, verbose=0)[0]
                att_matrix[(note_index-len(att_prediction)+sequence_length):(note_index+sequence_length), note_index] = att_prediction

            new_note = np.zeros(128)

            for idx, n_i in enumerate(notes_prediction[0]):
                try:
                    note_name = int_to_note[idx]
                    midi_note = note.Note(note_name)
                    new_note[midi_note.pitch.midi] = n_i

                except:
                    pass

            overall_preds.append(new_note)


            i1 = sample_with_temp(notes_prediction[0], notes_temp)
            i2 = sample_with_temp(durations_prediction[0], duration_temp)


            note_result = int_to_note[i1]
            duration_result = int_to_duration[i2]

            prediction_output.append([note_result, duration_result])

            notes_input_sequence.append(i1)
            durations_input_sequence.append(i2)

            if len(notes_input_sequence) > max_seq_len:
                notes_input_sequence = notes_input_sequence[1:]
                durations_input_sequence = durations_input_sequence[1:]
            if note_result == 'START':
                break
        overall_preds = np.transpose(np.array(overall_preds)) 

        output_folder = os.path.join(run_folder, 'output')

        midi_stream = stream.Stream()

        for pattern in prediction_output:
            note_pattern, duration_pattern = pattern
            if ('.' in note_pattern):
                notes_in_chord = note_pattern.split('.')
                chord_notes = []
                for current_note in notes_in_chord:
                    new_note = note.Note(current_note)
                    new_note.duration = duration.Duration(duration_pattern)
                    new_note.storedInstrument = instrument.Violoncello()
                    chord_notes.append(new_note)
                new_chord = chord.Chord(chord_notes)
                midi_stream.append(new_chord)
            elif note_pattern == 'rest':
                new_note = note.Rest()
                new_note.duration = duration.Duration(duration_pattern)
                new_note.storedInstrument = instrument.Violoncello()
                midi_stream.append(new_note)
            elif note_pattern != 'START':
                new_note = note.Note(note_pattern)
                new_note.duration = duration.Duration(duration_pattern)
                new_note.storedInstrument = instrument.Violoncello()
                midi_stream.append(new_note)
        midi_stream = midi_stream.chordify()
        midi_stream.write('midi', fp=os.path.join(output_folder, f'exp372_{i}.mid'))
        print("-"*40,f"{i}","-"*40)
        if i==1:
            break
        i+=1
    except:
        continue

2021-12-06 19:32:45.952798: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2021-12-06 19:32:45.972407: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 3000000000 Hz


---------------------------------------- 1 ----------------------------------------
